In [17]:
import pandas as pd
import numpy as np

# Carregar os dados
url = "../data/raw/Titanic-Dataset.csv"
data = pd.read_csv(url)


In [20]:

# Configurar pandas para mostrar 2 casas decimais
pd.set_option('display.precision', 2)

# =============================================
# 1. ANÁLISE EXPLORATÓRIA APRIMORADA
# =============================================
print("\n=== ANÁLISE EXPLORATÓRIA COMPLETA ===")

# 1.1. Análise de Cabines
print("\n--- Análise das Cabines por Classe ---")
data['Cabin_Letter'] = data['Cabin'].str[0].fillna('Unknown')
print("\nCabines registradas por classe:")
print(data.groupby('Pclass')['Cabin'].count())
print("\nLetras de cabine por classe:")
print(data.groupby(['Pclass', 'Cabin_Letter']).size().unstack(fill_value=0))

# 1.2. Análise de Títulos
print("\n--- Análise de Títulos ---")
data['Title'] = data['Name'].str.extract(r',\s([A-Za-z]+)\.', expand=False)
common_titles = ['Mr', 'Mrs', 'Miss', 'Master']
data['Title'] = data['Title'].apply(lambda x: x if x in common_titles else 'Rare')
print("\nDistribuição de títulos:")
print(data['Title'].value_counts())

# 1.3. Análise Familiar
print("\n--- Análise Familiar ---")
data['Family_Size'] = data['SibSp'] + data['Parch'] + 1
data['Is_Alone'] = (data['Family_Size'] == 1).astype(int)
print("\nSobrevivência por tamanho de família:")
print(data.groupby('Family_Size')['Survived'].mean().round(2))



=== ANÁLISE EXPLORATÓRIA COMPLETA ===

--- Análise das Cabines por Classe ---

Cabines registradas por classe:
Pclass
1    176
2     16
3     12
Name: Cabin, dtype: int64

Letras de cabine por classe:
Cabin_Letter   A   B   C   D   E  F  G  T  Unknown
Pclass                                            
1             15  47  59  29  25  0  0  1       40
2              0   0   0   4   4  8  0  0      168
3              0   0   0   0   3  5  4  0      479

--- Análise de Títulos ---

Distribuição de títulos:
Title
Mr        517
Miss      182
Mrs       125
Master     40
Rare       27
Name: count, dtype: int64

--- Análise Familiar ---

Sobrevivência por tamanho de família:
Family_Size
1     0.30
2     0.55
3     0.58
4     0.72
5     0.20
6     0.14
7     0.33
8     0.00
11    0.00
Name: Survived, dtype: float64


In [21]:
# =============================================
# 2. PRÉ-PROCESSAMENTO COMPLETO
# =============================================
def prepare_enhanced_data(df):
    # 2.1. Tratar dados faltantes
    df['Age'] = df.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.median()))
    df['Embarked'] = df['Embarked'].fillna('S')
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    
    # 2.2. Feature Engineering
    df['Has_Cabin'] = df['Cabin'].notna().astype(int)
    df['Family_Size'] = df['SibSp'] + df['Parch'] + 1
    df['Is_Alone'] = (df['Family_Size'] == 1).astype(int)
    
    # 2.3. Discretização
    df['Age_Group'] = pd.cut(df['Age'], bins=[0, 12, 18, 60, 100], 
                            labels=['Child', 'Teen', 'Adult', 'Elderly'])
    df['Fare_Group'] = pd.qcut(df['Fare'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    # 2.4. Tratar outliers
    fare_upper_limit = df['Fare'].quantile(0.95)
    df['Fare'] = np.where(df['Fare'] > fare_upper_limit, fare_upper_limit, df['Fare'])
    
    # 2.5. Converter variáveis categóricas
    df = pd.get_dummies(df, columns=[
        'Sex', 'Embarked', 'Pclass', 'Title', 'Age_Group', 'Fare_Group'
    ], drop_first=True)
    
    # 2.6. Selecionar features
    features = [
        'Age', 'SibSp', 'Parch', 'Fare', 'Has_Cabin', 'Family_Size', 'Is_Alone',
        'Sex_male', 'Embarked_Q', 'Embarked_S', 
        'Pclass_2', 'Pclass_3',
        'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare',
        'Age_Group_Teen', 'Age_Group_Adult', 'Age_Group_Elderly',
        'Fare_Group_Q2', 'Fare_Group_Q3', 'Fare_Group_Q4'
    ]
    
    # 2.7. Verificar dados faltantes
    print("\nValores faltantes após limpeza:")
    print(df[features].isnull().sum())
    
    return df[features], df['Survived']

In [22]:
# =============================================
# 3. APLICAÇÃO E VERIFICAÇÃO
# =============================================
# Aplicar pré-processamento
X_enhanced, y_enhanced = prepare_enhanced_data(data.copy())

# Verificar dados finais
print("\nDados aprimorados para modelagem (5 primeiras linhas):")
print(X_enhanced.head())

# Mostrar taxa de sobrevivência por novas features
print("\nTaxa de sobrevivência por título:")
print(data.groupby('Title')['Survived'].mean().round(2))


Valores faltantes após limpeza:
Age                  0
SibSp                0
Parch                0
Fare                 0
Has_Cabin            0
Family_Size          0
Is_Alone             0
Sex_male             0
Embarked_Q           0
Embarked_S           0
Pclass_2             0
Pclass_3             0
Title_Miss           0
Title_Mr             0
Title_Mrs            0
Title_Rare           0
Age_Group_Teen       0
Age_Group_Adult      0
Age_Group_Elderly    0
Fare_Group_Q2        0
Fare_Group_Q3        0
Fare_Group_Q4        0
dtype: int64

Dados aprimorados para modelagem (5 primeiras linhas):
    Age  SibSp  Parch   Fare  Has_Cabin  Family_Size  Is_Alone  Sex_male  \
0  22.0      1      0   7.25          0            2         0      True   
1  38.0      1      0  71.28          1            2         0     False   
2  26.0      0      0   7.92          0            1         1     False   
3  35.0      1      0  53.10          1            2         0     False   
4  35.0     